<a href="https://colab.research.google.com/github/publictrain/kubota/blob/main/kubotasoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import time
import random
import csv
import urllib
import os
import pprint
import time
import urllib.error
import urllib.request

# Colabでもどれが使ったライブラリかわかるようにしてくれ

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"}

def main():
  ans_list = []
  ans_list_box = []
  pdf_list = []
  k=int(input("(ここに入力した数かける30くらいの安保理データが手に入るよ！"))

  for i in range(0,k):#1ページ目からスタート
    load_url = "https://www.securitycouncilreport.org/un-documents-all-pdfs-last-added/page/{0}".format(i)# これヘッダーがないと動かない。スクレイピングよけだろう...
    html = requests.get(load_url,headers=headers)
    soup = BeautifulSoup(html.content, "html.parser")
    elems = soup.select("h7 > a")# elemsの型はBs特有のもの、h7 > aでネストの探索
    for elem in elems:
      if "pdf" in str(elem).lower():# 議事録のファイル名PVかpvで統制とって国際連合
        if "pv" in str(elem).lower():# pvからPVが議事録らしい
          elem_extract = str(elem.get("href"))# encodeしたままだとSTR型じゃなくてByte型で返してくる
          elem_encode = elem_extract.replace(" ","").encode('ascii', "ignore").decode('utf-8')# encode decodeはエラー起きやすい
          # print(type(elem_encode))
          pdf_list.append(elem_encode)#　elemの型はBs特有のもの、それをキャストしてる
    wait_xs()

  for p in pdf_list:
    print(p)

  for j in range(len(pdf_list)):
    req = urllib.request.Request(pdf_list[j], headers=headers)# 403よけHeader
    download_file(req, "/content/drive/MyDrive/Colab Notebooks/kubota/gomi/pekopeko{0}.pdf".format(j))


def wait_xs():
  sec = random.uniform(0.1, 0.5)
  time.sleep(sec)

def download_file(req, dst_path):
    try:
        with urllib.request.urlopen(req) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)



if __name__ == "__main__":
    main()



(ここに入力した数かける30くらいの安保理データが手に入るよ！10
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_PV.9038.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_PV_9035.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_pv.9033.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_pv.9032.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_pv.9029.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_pv.9027.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/S_PV.9021.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/N2232578.pdfN2232578.pdf
https://www.securitycouncilreport.org/atf/cf/%7B65BFCF9B-6D27-4E9C-8CD3-CF6E4FF96FF9%7D/s_pv.9018.pdf
https://www.securitycouncilreport.org

In [1]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 29.1 MB/s 
     |████████████████████████████████| 4.0 MB 49.0 MB/s 


In [13]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
import os
import csv

'''
なんでPythonにクラスがあんだよ
オブジェクト指向はどうなってんだオブジェクト指向は
お前ら禁じられた記法を
平気で使ってんじゃねえか
分かってんのか！？
「シン」が生まれたのは
人間が機械に甘えたせいだろうが
金取んのかよ！？
くそったれ！
'''


DIR = '/content/drive/MyDrive/Colab Notebooks/kubota/gomi/'
dir_num = sum(os.path.isfile(os.path.join(DIR, name)) for name in os.listdir(DIR))# ディレクトリのなかのファイル数求めてる
pekopeko_list = []

print(dir_num)

dir_num = 5 # これはデバッグ用

for i in range(dir_num):
  pdf_file_path = "/content/drive/MyDrive/Colab Notebooks/kubota/gomi/pekopeko{0}.pdf".format(i)

  with open(pdf_file_path , "rb") as pdf_file: #ファイルオブジェクトを受け取り、変数「pdf_file」に代入
      output = StringIO() #コンソールに出力されたテキストを取得するため、IOクラス「StringIO」使用
      resource_manager = PDFResourceManager()
      laparams = LAParams() #レイアウトの変更がなければデフォルトのままで 
      text_converter = TextConverter(resource_manager, output, laparams=laparams)
      page_interpreter = PDFPageInterpreter(resource_manager, text_converter)

      for i_page in PDFPage.get_pages(pdf_file): #1ベージずづ処理 
          page_interpreter.process_page(i_page)

      output_text = output.getvalue()
      output.close()
      text_converter.close()
  # print(output_text)
  pekopeko_list.append(output_text)

for pekopeko in pekopeko_list:
  print(pekopeko[:500])# デバッグ用

f = open("/content/drive/MyDrive/Colab Notebooks/kubota/gomi/pekopeko.csv", mode="w")

writer = csv.writer(f,lineterminator='\n')
for data in pekopeko_list:
    writer.writerow(data)
f.close()
# ここを二次元配列にしないと行けない気がする
# またあした（ねむい）2022/06/07



1338
United Nations
Security Council
Seventy-seventh year

9038th meeting
Friday, 20 May 2022, 10.15 a.m. 
New York

S/PV.9038

Provisional

President: Mrs. Thomas-Greenfield . . . . . . . . . . . . . . . . . . . . . . . . . .

(United States of America)

Members:

Albania  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Mr. Hoxha
Brazil . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Mr. Costa Filho
China . . . . . . . . . . . . . . . .
United Nations
Security Council
Seventy-seventh year

9035th meeting
Wednesday, 18 May 2022, 10 a.m. 
New York

S/PV.9035

Provisional

President: Mrs. Thomas-Greenfield/Mr. Mills . . . . . . . . . . . . . . . . . .

(United States of America)

Members:

Albania  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Ms. Dautllari
Brazil . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Mr. De Almeida Filho
China . . . . . . . . . . . 